# 🎯 Simulation 2: Sadece Model Eğitimi

Kalibrasyon ve skorlama olmadan sadece model eğitimi.

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

try:
    from risk_pipeline.pipeline16 import RiskModelPipeline, Config
except:
    import sys
    sys.path.append('..')
    from src.risk_pipeline.pipeline16 import RiskModelPipeline, Config

In [ ]:
# Eğitim verisi oluştur
np.random.seed(123)

n = 5000
train_df = pd.DataFrame({
    'app_id': [f'ID_{i:05d}' for i in range(n)],
    'app_dt': pd.date_range('2023-01-01', periods=n, freq='D'),
    'target': np.random.binomial(1, 0.15, n),
    
    # Numeric features
    'age': np.random.randint(18, 70, n),
    'income': np.random.lognormal(10, 0.5, n),
    'credit_score': np.random.normal(650, 100, n).clip(300, 850),
    'debt_ratio': np.random.uniform(0, 1, n),
    'loan_amount': np.random.exponential(50000, n),
    
    # Categorical features
    'education': np.random.choice(['HS', 'BSc', 'MSc', 'PhD'], n),
    'employment': np.random.choice(['Employed', 'Self', 'Retired'], n),
    'property': np.random.choice(['Own', 'Rent', 'Other'], n)
})

print(f"Training data: {train_df.shape}")
print(f"Target rate: {train_df['target'].mean():.2%}")

In [ ]:
# Sadece eğitim config'i
cfg = Config(
    id_col='app_id',
    time_col='app_dt',
    target_col='target',
    
    # Kalibrasyon YOK
    calibration_df=None,
    calibration_data_path=None,
    
    # Split ayarları
    use_test_split=True,
    test_size_row_frac=0.2,
    oot_window_months=2,
    
    # Model ayarları
    cv_folds=5,
    hpo_timeout=60,
    hpo_n_trials=10,
    
    # Output
    output_folder='outputs_training_only',
    output_excel_path='training_only_report.xlsx',
    write_artifacts=True  # Model dosyalarını kaydet
)

print("Configuration ready")

In [ ]:
# Pipeline çalıştır
pipeline = RiskModelPipeline(cfg)
pipeline.run(train_df)

print("\n" + "="*60)
print("Training Complete!")
print("="*60)
print(f"Best Model: {pipeline.best_model_name_}")
print(f"Final Features: {len(pipeline.final_vars_)}")
print(f"Calibrator: {'Yes' if pipeline.calibrator_ else 'No'}")

In [ ]:
# Model performansı
if pipeline.models_summary_ is not None:
    print("\nModel Performance:")
    print(pipeline.models_summary_[['model', 'auc_oot', 'gini_oot', 'ks_oot']])

In [ ]:
# Model dosyalarını kontrol et
import os
from pathlib import Path

output_path = Path(cfg.output_folder)
print("\nSaved Files:")
for file in output_path.glob(f"*{pipeline.cfg.run_id}*"):
    size_kb = os.path.getsize(file) / 1024
    print(f"  • {file.name} ({size_kb:.1f} KB)")